In [17]:
# Libraries needed for basic web-scraping
from IPython.core.display import HTML
from bs4 import BeautifulSoup, NavigableString, Tag
from IPython.display import IFrame
from urllib.request import urlopen, Request
import pandas as pd 
import urllib
import os
import re
import traceback

In [18]:
# Scrape data from website
target_url='https://coinatmradar.com/country/226/bitcoin-atm-united-states/'
# fool the server :/
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

req = Request(url=target_url, headers=headers) 
coinatmradar_usa_html = urlopen(req).read() 

# # Saving scraped HTML to .html file (for later processing)
# with open('coinatmradar_usa.html', 'w') as f:
#     f.write(coinatmradar_usa_html)

# Use html.parser to create soup
s = BeautifulSoup(coinatmradar_usa_html, 'html.parser')
print(s.prettify())


<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-US" prefix="og: http://ogp.me/ns#">
 <!--<![endif]-->
 <head>
  <title>
   Bitcoin ATM United States – find bitcoin machine locations
  </title>
  <meta content="Locations of Bitcoin ATM in United States The easiest way to buy and sell bitcoins." name="description"/>
  <meta charset="utf-8"/>
  <link href="/favicon.ico?v=2" rel="shortcut icon" type="image/x-icon"/>
  <meta content="#2790a1" name="theme-color">
   <meta content="app-id=944855781" name="apple-itunes-app"/>
   <meta content="app-id=com.coinatmradar.android" name="google-play-app"/>
   <link href="/img/apple-touch-icon.png" rel="apple-touch-icon"/>
   <link href="/img/apple-touch-icon.png" rel="apple-touch-icon-precomposed"/>
   <link href="/css/g

In [ ]:
cities = s.find_all('div', {'class': 'third'})
i = 0

city_venue_cnts = []
for column in cities: 
    cur_state = ''
    for elem in column:
        if isinstance(elem, Tag):
            if 'class' in elem.attrs:
                cur_state = elem.text.split(',')[0]
                continue

            city_name = elem.a.text
            tmp_str_arr = elem.text.split(' ')
            venue_count = tmp_str_arr[len(tmp_str_arr)-1]
            try: 
                venue_count = int(venue_count)
            except ValueError as e:
                print(e)
                continue
            
            city_venue_cnt = {}
            city_venue_cnt['city']  = city_name
            city_venue_cnt['state'] = cur_state
            city_venue_cnt['count'] = venue_count
            city_venue_cnts.append(city_venue_cnt)

# remove duplicate value for Birmingham            
city_venue_cnts = city_venue_cnts[1:]
city_venue_cnts[0]['city'] = city_venue_cnts[0]['city'].split(',')[0]
df_venues_by_city = pd.json_normalize(city_venue_cnts)
# TODO: need to change this relative path when code is moved 
df_venues_by_city.to_csv('../data/interim/ATM_COUNT_BY_CTIY', index=False)

print(df_venues_by_city.shape)
for cnt in city_venue_cnts:
    print(cnt)

In [14]:

def containsDigit(variable):
    return any(char.isdigit() for char in variable)

def containsMoneySign(variable):
    return '$' in variable

def stripNonNumeric(variable):
    return re.sub("[^0-9^.]", "", variable)

def stripNonNumericAndDecimal(variable):
    return re.sub("[^0-9]", "", variable)

# What data can we get? 

html = ''
with open(f'../data/raw/city_data/city-data-Alexandria-Virginia.html', 'r') as f:
            html = f.read()

city_data = dict() 
        

class city_data_html_extractor():
    def __init__(self):
        self.general_demographic_data_all = []
        self.racial_data_all = []
        self.industries_and_occupations_data_all = []
        self.food_env_data_all = []
        self.extraction_errors = 0
    
    def exception_handler(e):
        print(e)
        self.extraction_errors += 1
        
    def get_all_data(self, s, df_index):
        all_data_dict = {
            'general_demographic_data_row': self.get_general_demographic_data(s, df_index),
            'race_data_row': self.get_racial_data(s, df_index),
            'industry_data_row': self.get_industries_and_occupations_data(s, df_index)['industry_dict'],
            'occupation_data_row': self.get_industries_and_occupations_data(s, df_index)['occupation_dict'],
            'food_env_data_row': self.get_food_env_data(s, df_index),
            'expenditure_data_row': self.get_city_expenditure_data(s, df_index),
            'debt_data_row': self.get_city_debt_data(s, df_index),
            'assets_data_row': self.get_city_assets_data(s, df_index),
            'revenue_data_row': self.get_city_revenue_dict(s, df_index)
        }
        return all_data_dict
        
    def get_general_demographic_data(self, s, df_index):
        city_general_demographic_data = dict(df_index)
        city_general_demographic_data.update(self.get_population_data(s, df_index) or {})
        city_general_demographic_data.update(self.get_median_age_data(s, df_index) or {})
        city_general_demographic_data.update(self.get_median_income_data(s, df_index) or {})
        city_general_demographic_data.update(self.get_median_rent_data(s, df_index) or {})
        city_general_demographic_data.update(self.get_cost_of_living_data(s, df_index) or {})
        city_general_demographic_data.update(self.get_pop_density_data(s, df_index) or {})
        return city_general_demographic_data

    def get_population_data(self, s, df_index):
        try: 
            city_population_data = s.find('section', {'id': 'city-population'})
            city_general_demographic_data = dict(df_index)
            for elem in city_population_data: 
                if isinstance(elem, NavigableString):
                    pop_data_arr =  elem.split(' ')
                    pop_data_arr = list(filter(containsDigit, pop_data_arr))
                    try:
                        pop_data_arr = list(map(stripNonNumeric, pop_data_arr))
                        city_general_demographic_data['total population 2019'] = int(stripNonNumericAndDecimal(pop_data_arr[0]))
                        if (len(pop_data_arr)>=3):
                            city_general_demographic_data['percent urban 2019'] = pop_data_arr[1]
                            city_general_demographic_data['percent rural 2019'] = pop_data_arr[2]
                        return city_general_demographic_data
                    except IndexError as e:
                        print('<get_population_data>',e)
                        print('could not locate all data for population for', df_index['city'])
                        self.extraction_errors += 1
                        return city_general_demographic_data
        except Exception as e:
            print(e)
            print(traceback.format_exc())
            self.extraction_errors += 1
            


    def get_median_age_data(self, s, df_index):
        median_age_data = s.find('section', {'id': 'median-age'})
        try:
            city_general_demographic_data = dict(df_index)
            for elem in median_age_data:
                idx = elem.text.find(':')
                median_resident_age = elem.text[idx+1:idx+5] # median age is like 34.4, for example
                city_general_demographic_data['median age'] = float(median_resident_age)
            return city_general_demographic_data
        except Exception as e:
            print('<get_median_age_data>',e)
            print(traceback.format_exc())
            print(median_age_data)
            print(df_index)
            self.extraction_errors += 1
            
    def get_median_income_data(self, s, df_index):
        try:
            median_income_data = s.find('section', {'id': 'median-income'})
            city_general_demographic_data = dict(df_index)
            for elem in median_income_data:
                if isinstance(elem, Tag):
                    if 'Estimated per capita income' in elem.text:
                        income_arr = list(filter(containsDigit, elem.text.splitlines() ))
                        for elem in income_arr:
                            if 'Estimated per capita income in 2019' in elem:
                                temp_str = elem.split(':')[1]
                                temp_str_arr = temp_str.split('(')
                                temp_str = temp_str_arr[0]
                                median_income = stripNonNumeric(temp_str)
                                city_general_demographic_data['median income 2019'] = float(median_income)
                            if 'Estimated median house or condo value in 2019' in elem:
                                temp_str = elem.split(':')[1]
                                temp_str_arr = temp_str.split('(')
                                temp_str = temp_str_arr[0]
                                median_house_price = stripNonNumeric(temp_str)
                                city_general_demographic_data['median house price 2019'] = float(median_house_price)
            return city_general_demographic_data
        except Exception as e:
            print('<get_median_income_data>',e)
            self.extraction_errors += 1


    def get_median_rent_data(self, s, df_index):
        try:
            median_rent_data = s.find('section', {'id': 'median-rent'})
            city_general_demographic_data = dict(df_index)
            for elem in median_rent_data:
                if isinstance(elem, Tag):
                    #print('elem:',elem.text)
                    median_rent_arr = elem.text.split(':')
                    median_rent = int(stripNonNumericAndDecimal(median_rent_arr[1]))
                    city_general_demographic_data['median rent 2019'] = float(median_rent)
            return city_general_demographic_data
        except Exception as e:
            print('<get_median_rent_data>',e)
            self.extraction_errors += 1
            
    def get_cost_of_living_data(self, s, df_index):
        try:
            cost_of_living_index_data = s.find('section', {'id': 'cost-of-living-index'})
            city_general_demographic_data = dict(df_index)
            for elem in cost_of_living_index_data:
                if isinstance(elem, NavigableString):
                    cost_of_living_index = float(elem)
                    city_general_demographic_data['cost of living index'] = float(cost_of_living_index)
            return city_general_demographic_data
        except Exception as e:
            print('<get_cost_of_living_data>',e)
            self.extraction_errors += 1
            
    def get_pop_density_data(self, s, df_index):
        try:
            population_density_data = s.find('section', {'id': 'population-density'})
            city_general_demographic_data = dict(df_index)
            for elem in population_density_data:
                if isinstance(elem, Tag):
                    if 'Population density' in elem.text:
                        pop_density = stripNonNumericAndDecimal(elem.text)
                        city_general_demographic_data['population density 2019'] = float(pop_density)            
            return city_general_demographic_data
        except Exception as e:
            print('<get_pop_density_data>',e)
            self.extraction_errors += 1

    def get_racial_data(self, s, df_index):
        try:
            city_racial_data = dict(df_index)
            racial_brkdwn_data = s.find('section', {'id': 'races-graph'})
            for elem in racial_brkdwn_data:
                if isinstance(elem, Tag):
                    cracial_brkdwn_data_arr = str.splitlines(elem.text)
                    cracial_brkdwn_data_arr =  list(filter(stripNonNumeric,cracial_brkdwn_data_arr))
                    for elem in cracial_brkdwn_data_arr:
                        elem_arr = elem.split('%')
                        data = elem_arr[0] # numeric values
                        key = elem_arr[1]
                        idx = elem.rfind(',')

                        # This is true when the total count is atleast 1000
                        if idx != -1:
                            city_racial_data[key] = float(data[idx+4:])
                        # This is true when the total count is less than 1000
                        else:
                            city_racial_data[key] = float(data[-3:])
            return city_racial_data
        except Exception as e:
            print('<get_racial_data>',e)
            self.extraction_errors += 1

    
    def get_industries_and_occupations_data(self, s, df_index):
        try:
            industries_and_occupations_data = s.find('section', {'id': 'most-common-industries'})
            industry_dict = dict(df_index)
            occupation_dict = dict(df_index)
            for elem in industries_and_occupations_data:
                if isinstance(elem, Tag):
                    industries_and_occupation_arr_raw = str.splitlines(elem.text)
                    # Apply lambda function to return only elems with a positive length, then caset to list
                    industries_and_occupation_arr = list(filter(lambda x: len(x) > 0, industries_and_occupation_arr_raw))
                    industries_arr = []
                    occupation_arr = []
                    # Array contains information on both industries and occupations, so we'll
                    # need to splice at a convienent index... html hell :(
                    for idx, elem in enumerate(industries_and_occupation_arr):
                        if 'Most common occupations' in elem:
                            industries_arr = industries_and_occupation_arr[0:idx]
                            occupation_arr = industries_and_occupation_arr[idx:]
                            break
                    #print(industries_and_occupation_arr, "\n\n")

                    # We don't want elements that don't contain numeric values
                    industries_arr = list(filter(lambda x: any(char.isdigit() for char in x), industries_arr))
                    occupation_arr = list(filter(lambda x: any(char.isdigit() for char in x), occupation_arr))
                    #print(industries_arr)
                    #print(occupation_arr)

                    for elem in industries_arr:
                        elem_str_arr = elem.split('(')
                        #print(elem_str_arr)
                        category =  elem_str_arr[0][:len(elem_str_arr[0])-1]
                        value = elem_str_arr[1].replace('%','').replace(')','')
                        #print(value)
                        #print(category)
                        industry_dict[category.lower()] = float(value)

                    for elem in occupation_arr:
                        elem_str_arr = elem.split('(')
                        #print(elem_str_arr)
                        category =  elem_str_arr[0][:len(elem_str_arr[0])-1]
                        value = elem_str_arr[1].replace('%','').replace(')','')
                        #print(value)
                        #print(category)
                        occupation_dict[category.lower()] = float(value)
            return {'industry_dict': industry_dict, 'occupation_dict': occupation_dict}
        except Exception as e:
            print('<get_industries_and_occupations_data>',e)
            self.extraction_errors += 1
    
    def get_food_env_data(self, s,df_index):
        try:
            food_environment_data = s.find('section', {'id': 'food-environment'})
            food_env_dict = dict(df_index) 
            for elem in food_environment_data:
                if isinstance(elem, Tag):
                    food_env_data_str = str(elem.text)
                    if 'Number of' in food_env_data_str:
                        category = food_env_data_str.split(':')[0].replace('Number of', '')
                        
                        # Basically we need to try a number of different strings to 
                        # begin the index, since webpages data is perfected normalized
                        idx = food_env_data_str.lower().find('County:'.lower())
                        if idx != -1:
                            start_idx = idx + len('County:')
                            value = food_env_data_str[start_idx:start_idx+4]
                            food_env_dict[category[1:] + ' per 10k'] = float(value)
                            continue
                        
                        idx = food_env_data_str.lower().find('Here:'.lower())
                        if idx != -1:
                            start_idx = idx + len('Here:')
                            value = food_env_data_str[start_idx:start_idx+4]
                            food_env_dict[category[1:] + ' per 10k'] = float(value)
                            continue                          
                            
                        cur_city = df_index['city']
                        city_string = f'{cur_city} city'
                        idx = food_env_data_str.lower().find(city_string.lower())
                        if idx != -1:
                            start_idx = idx + len(city_string)
                            value = food_env_data_str[start_idx:start_idx+4]
                            food_env_dict[category[1:] + ' per 10k'] = float(value)
                            continue     
                        
                        print('Did not find target string "County:" in string...')
                        print(food_env_data_str, "\n\n")
                    else:
                        continue
            return food_env_dict
        except Exception as e:
            print('<get_food_env_data>',e)
            self.extraction_errors += 1

    def get_city_expenditure_data(self, s, df_index):
        try:
            govFinances_expenditure_data = s.find('div', {'id': 'govFinancesE'})
            fin_expenditure_dict = dict(df_index)
            for elem in govFinances_expenditure_data:
                if isinstance(elem, Tag):
                    #print('elem:',elem.text)
                    revenue_data_arr = str.splitlines(elem.text)
                    revenue_data_arr = list(filter(containsDigit, revenue_data_arr))
                    revenue_data_arr = list(filter(containsMoneySign, revenue_data_arr))
                    for elem in revenue_data_arr:
                        key = elem.split(':')[0]
                        value = elem.split(':')[1]
                        value = value.split('(')[1].split(')')[0]
                        value = stripNonNumeric(value)
                        fin_expenditure_dict[key] = value
            return fin_expenditure_dict
        except Exception as e:
            print('<get_city_expenditure_data>',e)
            self.extraction_errors += 1

    def get_city_debt_data(self, s,df_index):
        try:
            govFinances_debt_data = s.find('div', {'id': 'govFinancesD'})
            fin_debt_dict = dict(df_index)  
            for elem in govFinances_debt_data:
                if isinstance(elem, Tag):
                    #print('elem:',elem.text)
                    debt_data_arr = str.splitlines(elem.text)
                    debt_data_arr = list(filter(containsDigit, debt_data_arr))
                    debt_data_arr = list(filter(containsMoneySign, debt_data_arr))
                    for elem in debt_data_arr:
                        key = elem.split(':')[0]
                        value = elem.split(':')[1]
                        value = value.split('(')[1].split(')')[0]
                        value = stripNonNumeric(value)
                        fin_debt_dict[key] = value
            return fin_debt_dict
        except Exception as e:
            print('<get_city_debt_data>',e)
            self.extraction_errors += 1
          

    def get_city_assets_data(self, s,df_index):
        try: 
            govFinances_assets_data = s.find('div', {'id': 'govFinancesC'})
            fin_assets_dict = dict(df_index)
            for elem in govFinances_assets_data:
                if isinstance(elem, Tag):
                    #print('elem:',elem.text)
                    assets_data_arr = str.splitlines(elem.text)
                    assets_data_arr = list(filter(containsDigit, assets_data_arr))
                    assets_data_arr = list(filter(containsMoneySign, assets_data_arr))
                    for elem in assets_data_arr:
                        key = elem.split(':')[0]
                        value = elem.split(':')[1]
                        value = value.split('(')[1].split(')')[0]
                        value = stripNonNumeric(value)
                        fin_assets_dict[key] = value
            return fin_assets_dict
        except Exception as e:
            print('<get_city_assets_data>',e)
            self.extraction_errors += 1

    def get_city_revenue_dict(self, s,df_index):
        try: 
            govFinances_revenue_data = s.find('div', {'id': 'govFinancesR'})
            fin_revenue_dict = dict(df_index)
            for elem in govFinances_revenue_data:
                if isinstance(elem, Tag):
                    #print('elem:',elem.text)
                    revenue_data_arr = str.splitlines(elem.text)
                    revenue_data_arr = list(filter(containsDigit, revenue_data_arr))
                    revenue_data_arr = list(filter(containsMoneySign, revenue_data_arr))
                    for elem in revenue_data_arr:
                        key = elem.split(':')[0]
                        value = elem.split(':')[1]
                        value = value.split('(')[1].split(')')[0]
                        value = stripNonNumeric(value)
                        fin_revenue_dict[key] = value
            return fin_revenue_dict
        except Exception as e:
            print('<get_city_revenue_dict>',e)
            self.extraction_errors += 1
            
html_extractor = city_data_html_extractor()

# # commuters_data = s.find('section', {'id': 'commuters'})

# education_info_data = s.find('section', {'id': 'most-common-industries'})

# def get_to_state_comparision_data(s):
#     to_state_comparisons = s.find('section', {'id': 'comparisons'})
#     for elem in to_state_comparisons:
#         if isinstance(elem, Tag):
#             #print('elem:',elem.text)
#             compare_data_arr = str.splitlines(elem.text)
#             print(len(compare_data_arr))
#             print(compare_data_arr, "\n\n")
        
        
        
# fcc_licenses_data = s.find('section', {'id': 'fcc-licenses'})
# for elem in fcc_licenses_data:
#     if isinstance(elem, Tag):
#         #print('elem:',elem.text)
#         tech_infra_data_arr = str.splitlines(elem.text)
#         print(tech_infra_data_arr, "\n\n")


# maybe
# table id='H09'

s = BeautifulSoup(html, 'html.parser')

def get_to_state_comparision_data(s):
    to_state_comparisons = s.find('section', {'id': 'comparisons'})
    for elem in to_state_comparisons:
        if isinstance(elem, Tag):
            compare_data_arr = str.splitlines(elem.text)
            if len(compare_data_arr) > 1:
                compare_data_arr = list(filter(lambda x: len(x) > 0, compare_data_arr))
                print(compare_data_arr, "\n\n")
            

get_to_state_comparision_data(s)


['Median house value above state average.', 'Unemployed percentage significantly below state average.', 'Hispanic race population percentage above state average.', 'Foreign-born population percentage significantly above state average.', 'Renting percentage above state average.', 'Length of stay since moving in below state average.', 'Number of rooms per house below state average.', "Percentage of population with a bachelor's degree or higher significantly above state average."] 




In [20]:

df_venues_by_city = pd.read_csv('../data/interim/ATM_COUNT_BY_CTIY.csv')
# this pipeline with extract a page from the city-data website
# for each city listed on coinatmradar.com. We will cash the 
# pages in data/raw/city_data, since this extaction is so 
# time-consuming 

def strip_state_abbrv(broken_string):
    city = broken_string.split(',')
    city = list(filter(lambda x: len(x) > 3, city))
    return city[0]


city_state_path_dict =  dict()
includes_state_abbrv = []

broken_urls = []
for index, row in df_venues_by_city.iterrows():
    try:
        city, state = '', ''
        
        if ',' in row['city']:
            city = strip_state_abbrv(row['city'])
        else:
            city = row['city']
            
        glue = '-'
        city = str(city).split(' ')
        city = glue.join(city)
        state = str(row['state']).split(' ')
        state = glue.join(state)

        # missing data for some Indiana cities 
        city_data_url = f'https://www.city-data.com/city/{city}-{state}.html'
        #print(city_data_url)

        file_path = f'../data/raw/city_data/city-data-{city}-{state}.html'
        if os.path.isfile(file_path):
            city_state_path_dict[f'city-data-{city}-{state}.html'] = {'city': city, 'state': state}
            #print('local copy of html page detected')
            continue
    
        r = urllib.request.urlopen(city_data_url) 

        site_content = r.read().decode('utf-8')

        # Saving scraped HTML to .html file (for later processing)
        with open(file_path, 'w') as f:
            f.write(site_content)
            
        # create mapping between filepath and city and state index 
        city_state_path_dict[f'city-data-{city}-{state}.html'] = {'city': city, 'state': state}

    except urllib.error.HTTPError as err:
        print(err)
        print('city:', city, 'state:', state)
        broken_urls.append(city_data_url)
        continue
    except AttributeError as err:
        print(err)
        broken_urls.append(city_data_url)
        continue
    

HTTP Error 404: Not Found
city: Aurora-CO state: Colorado
HTTP Error 404: Not Found
city: Saint-Pete-Beach state: Florida
HTTP Error 404: Not Found
city: LaFayette state: Georgia
HTTP Error 404: Not Found
city: Kailua-Kona state: Hawaii
HTTP Error 404: Not Found
city: Boise state: Idaho
HTTP Error 404: Not Found
city: Coeur-d'Alene state: Idaho
HTTP Error 404: Not Found
city: Aurora-IL state: Illinois
HTTP Error 404: Not Found
city: St-Charles state: Illinois
HTTP Error 404: Not Found
city: Carbondale state: Indiana
HTTP Error 404: Not Found
city: Springville state: Indiana
HTTP Error 404: Not Found
city: Bettendorf state: nan
HTTP Error 404: Not Found
city: Burlington state: nan
HTTP Error 404: Not Found
city: Cedar-Rapids state: nan
HTTP Error 404: Not Found
city: Council-Bluffs state: nan
HTTP Error 404: Not Found
city: Davenport state: nan
HTTP Error 404: Not Found
city: Decorah state: nan
HTTP Error 404: Not Found
city: Denison state: nan
HTTP Error 404: Not Found
city: Des-Moines

In [21]:
# manual work :(
broken_urls

['https://www.city-data.com/city/Aurora-CO-Colorado.html',
 'https://www.city-data.com/city/Saint-Pete-Beach-Florida.html',
 'https://www.city-data.com/city/LaFayette-Georgia.html',
 'https://www.city-data.com/city/Kailua-Kona-Hawaii.html',
 'https://www.city-data.com/city/Boise-Idaho.html',
 "https://www.city-data.com/city/Coeur-d'Alene-Idaho.html",
 'https://www.city-data.com/city/Aurora-IL-Illinois.html',
 'https://www.city-data.com/city/St-Charles-Illinois.html',
 'https://www.city-data.com/city/Carbondale-Indiana.html',
 'https://www.city-data.com/city/Springville-Indiana.html',
 'https://www.city-data.com/city/Bettendorf-nan.html',
 'https://www.city-data.com/city/Burlington-nan.html',
 'https://www.city-data.com/city/Cedar-Rapids-nan.html',
 'https://www.city-data.com/city/Council-Bluffs-nan.html',
 'https://www.city-data.com/city/Davenport-nan.html',
 'https://www.city-data.com/city/Decorah-nan.html',
 'https://www.city-data.com/city/Denison-nan.html',
 'https://www.city-data.c

In [ ]:

file_arr = [f for f in os.listdir('../data/raw/city_data/')]

for file in file_arr: 
    print(city_state_path_dict[file])

In [ ]:
df_dict = {
    "general_demographic_data": [],
    "race_data": [],
    "industry_data": [],
    "occupation_data": [],
    "food_env_data": [],
    "expenditure_data": [],
    "debt_data": [],
    "assets_data": [],
    "revenue_data": []
}

file_arr = [f for f in os.listdir('../data/raw/city_data/')]

for file in file_arr: 
    print(file, city_state_path_dict[file])
    
    html = ''
    with open(f'../data/raw/city_data/{file}', 'r') as f:
            html = f.read()
    
    s = BeautifulSoup(html, 'html.parser')
    all_data_for_city = html_extractor.get_all_data(s, df_index = city_state_path_dict[file])
    
    # All each dictionary (soon-to-be-row) to its respective array (soon-to-be-column). 
    # Basically, we move dicts from a dict of dicts to a dict of lists, each of which
    # contains dicts
    for key in all_data_for_city:
        df_dict_idx = key[:-4]
        df_dict[df_dict_idx].append(all_data_for_city[key])
        
# After we scraping each html page, use list comphrehension to 
# filter out None values in each arrary of dicts,
# then create the dataframe and save it to 'data/interim'
for key in df_dict:
    df_dict[key] = [x for x in df_dict[key] if x is not None]

    new_df = pd.json_normalize(df_dict[key])
    new_df.to_csv(f'../data/interim/{key}.csv', index=False)

    print('New DataFrame created. Relative path is:  ../../data/interim/{key}.csv')
    print('At a glance:\n')
    print('Shape: ', new_df.shape)
    print('Columns: ', new_df.dtypes)


In [ ]:
def get_to_state_comparision_data(s, df_index):
    to_state_comparisons = s.find('section', {'id': 'comparisons'})
    for elem in to_state_comparisons:
        if isinstance(elem, Tag):
            compare_data_arr = str.splitlines(elem.text)
            if len(compare_data_arr) > 1:
                compare_data_arr = list(filter(lambda x: len(x) > 0, compare_data_arr))
                print(compare_data_arr, "\n\n")
                
                compare_dict = dict(df_index)
                for idx, val in enumerate(compare_data_arr): # to make it more dynamic?
                    compare_dict['Row {}'.format(idx)] = val
                return compare_dict
                
file_arr = [f for f in os.listdir('../data/raw/city_data/')]

compare_arr = []
for file in file_arr: 
    print(file, city_state_path_dict[file])
    
    html = ''
    with open(f'../data/raw/city_data/{file}', 'r') as f:
            html = f.read()
    
    s = BeautifulSoup(html, 'html.parser')
    compare_arr.append(get_to_state_comparision_data(s, df_index=city_state_path_dict[file]))
            

In [37]:
compare_arr

compare_df = pd.json_normalize(compare_arr)

compare_df_copy = compare_df.copy()

In [33]:
compare_df.head()

,city,state,Row 0,Row 1,Row 2,Row 3,Row 4,Row 5,Row 6,Row 7,Row 8,Row 9,Row 10
0,Nevada-City,California,Median household income below state average.,Unemployed percentage below state average.,Black race population percentage significantly...,Hispanic race population percentage significan...,Median age significantly above state average.,Foreign-born population percentage significant...,Length of stay since moving in significantly a...,House age above state average.,Institutionalized population percentage signif...,Number of college students below state average.,NaN
1,Port-Jervis,New-York,Median house value below state average.,Unemployed percentage below state average.,Black race population percentage below state a...,Foreign-born population percentage significant...,Number of college students below state average.,NaN,NaN,NaN,NaN,NaN,NaN
2,Richmond,California,Unemployed percentage below state average.,Black race population percentage significantly...,Foreign-born population percentage above state...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kansas-City,Missouri,Unemployed percentage significantly below stat...,Black race population percentage above state a...,Hispanic race population percentage significan...,Foreign-born population percentage above state...,Length of stay since moving in below state ave...,Percentage of population with a bachelor's deg...,NaN,NaN,NaN,NaN,NaN
4,Annapolis,Maryland,Unemployed percentage significantly below stat...,Hispanic race population percentage above stat...,Renting percentage above state average.,Percentage of population with a bachelor's deg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
prefix_arr = [
"Median household income",
"Median house value",
"Unemployed percentage",
"Black race population percentage",
"Hispanic race population percentage",
"Median age",
"Foreign-born population percentage",
"House age",
"Number of college students",
"Renting percentage",
]




def compare_data (row, prefix):
    for column in compare_df.columns:
        #print(row[column])
        if not pd.isna(row[column]):
            if prefix in row[column]:
                return row[column].replace(prefix,'')

for prefix in prefix_arr:
    compare_df_copy[prefix] = compare_df_copy.apply(lambda row: compare_data(row, prefix), axis=1)

In [92]:
compare_df_copy.columns

Index(['city', 'state', 'Row 0', 'Row 1', 'Row 2', 'Row 3', 'Row 4', 'Row 5',
       'Row 6', 'Row 7', 'Row 8', 'Row 9', 'Row 10',
       'Black race population percentage', 'Median household income',
       'Median house value', 'Unemployed percentage',
       'Hispanic race population percentage', 'Median age',
       'Foreign-born population percentage', 'House age',
       'Number of college students', 'Renting percentage'],
      dtype='object')

In [94]:
compare_df_copy[prefix_arr]

,Median household income,Median house value,Unemployed percentage,Black race population percentage,Hispanic race population percentage,Median age,Foreign-born population percentage,House age,Number of college students,Renting percentage
0,below state average.,None,below state average.,significantly below state average.,significantly below state average.,significantly above state average.,significantly below state average.,above state average.,below state average.,None
1,None,below state average.,below state average.,below state average.,None,None,significantly below state average.,None,below state average.,None
2,None,None,below state average.,significantly above state average.,None,None,above state average.,None,None,None
3,None,None,significantly below state average.,above state average.,significantly above state average.,None,above state average.,None,None,None
4,None,None,significantly below state average.,None,above state average.,None,None,None,None,above state average.
5,None,None,below state average.,significantly below state average.,None,below state average.,significantly below state average.,None,above state average.,above state average.
6,None,None,below state average.,above state average.,above state average.,significantly below state average.,significantly above state average.,below state average.,significantly above state average.,None
7,below state average.,significantly below state average.,significantly below state average.,significantly below state average.,None,None,above state average.,significantly below state average.,None,None
8,None,None,significantly below state average.,None,None,significantly below state average.,above state average.,None,above state average.,None
9,None,None,below state average.,significantly below state average.,None,below state average.,below state average.,None,None,None
